In [1]:
import folium
from folium import plugins
import ipywidgets as widgets
import os
import json
from IPython.display import display, clear_output

# Latitude and Longitude coordinates
UMMlocation = (13.169595, 77.535095)

class Navigator:
    def __init__(self):
        self.geoResources = {}
        self.hospitalLocation = UMMlocation
        self.position = 'P_E'  # Default starting point
        self.destination = 'Ad'  # Default destination

        for root, dirs, files in os.walk('C:/Users/Keerthana/Downloads/Py-Nav/Py_Navigator/GeoResources'):
            for file in files:
                self.geoResources[file.split('.')[0]] = os.path.join(root, file)

    def changeDestination(self, newDestination):
        self.destination = newDestination
        self.redrawMap()

    def changeStartPoint(self, newStartPoint):
        self.position = newStartPoint
        self.redrawMap()

    def drawPathWay(self, hospitalMap):
        def switchPosition(coordinate):
            return [coordinate[1], coordinate[0]]

        searchString = self.position + "_" + self.destination
        if searchString not in self.geoResources:
            print(f"Error: Pathway {searchString} not found in geoResources")
            return

        with open(self.geoResources[searchString]) as f:
            testWay = json.load(f)

        for feature in testWay['features']:
            path = feature['geometry']['coordinates']

        finalPath = list(map(switchPosition, path))
        folium.plugins.AntPath(finalPath).add_to(hospitalMap)

    def drawBuilding(self, hospitalMap):
        hauseOutline = self.geoResources.get(self.destination)
        if hauseOutline:
            folium.GeoJson(hauseOutline, name="geojson").add_to(hospitalMap)
        else:
            print(f"Error: Building {self.destination} not found in geoResources")

    def redrawMap(self):
        hospitalMap = folium.Map(location=self.hospitalLocation, width="75%", zoom_start=17)
        self.drawPathWay(hospitalMap)
        self.drawBuilding(hospitalMap)
        with output:
            clear_output(wait=True)
            display(hospitalMap)

# Create the Navigator instance
myNavigator = Navigator()

# Mapping human-readable names to keys
start_point_map = {
    'Entrance': 'P_E',
    'Admission': 'P_E_Ad',
    'canteen': 'P_E_Cn',
    'Engineering': 'P_E_En',
    'Fountain': 'P_E_Fo'
}

destination_map = {
    'Admission': 'Ad',
    'Canteen': 'Ca',
    'Engineering Block': 'En',
    'Fountain': 'Fo',  
    'Managament Block': 'Mn', 
    'Parking': 'Pa', 
    'University-Pitch': 'Pi' 
}

# Create widgets
selectHouse_widget = widgets.Select(
    options=destination_map.keys(),
    value='Admission',
    description='Target',
    disabled=False
)

selectPosition_widget = widgets.Select(
    options=start_point_map.keys(),
    value='Entrance',
    description='Start',
    disabled=False
)

# Update function for both widgets
def updateMap(change=None):
    start_key = start_point_map[selectPosition_widget.value]
    target_key = destination_map[selectHouse_widget.value]
    myNavigator.changeStartPoint(start_key)
    myNavigator.changeDestination(target_key)

# Attach the update function to both widgets
selectPosition_widget.observe(updateMap, names='value')
selectHouse_widget.observe(updateMap, names='value')

# Create output widget for the map
output = widgets.Output()

# Display the widgets and the map
widget_box = widgets.HBox([selectPosition_widget, selectHouse_widget])
display(widget_box)
display(output)

# Initial map display
updateMap()


Output()